In [ ]:
! pip install openai==1.0.0
! pip install load_dotenv
! pip install gtts
! pip install gtts pydub
! pip install playsound
! pip install roboflow
!pip install pydub
! pip install pyttsx3

In [28]:
import openai
import os

from dotenv import load_dotenv
OPENAI_API_KEY = "Add OPEN-API Key"

# Load the environment variables
load_dotenv()

# Set the OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
%env OPENAI_API_KEY= OPENAI_API_KEY #Add API Key

In [30]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [ ]:
from roboflow import Roboflow
from IPython.display import Audio
import json
import pyttsx3
from gtts import gTTS

# Initialize Roboflow
rf = Roboflow(api_key="OPMeISeEpQl74oLVReUR")
project = rf.workspace().project("car_-w0bxi")
model = project.version(1).model

# Predict video
job_id, signed_url, _ = model.predict_video(
    "Road_Sign.mp4",
    fps=3,
    prediction_type="batch-video",
)
print("Signed URL", signed_url)

# Get video results
results = model.poll_until_video_results(job_id)
print("Results", results)

# Extract classes from predictions
classes = []

for frame_data in results['car_-w0bxi']:
    for prediction in frame_data['predictions']:
        class_name = prediction['class']
        classes.append(class_name)

# Initialize text-to-speech engine

# Initialize an array to store responses
responses_array = []

# Initialize a variable to keep track of the last printed response
last_printed_response = None

# Feed classes into OpenAI prompt
for class_name in classes:
    prompt = f"""
    Your task is to direct and give instructions to a driver as there is a {class_name} approaching ahead.

    Write a description based on the information provided only by not providing extra information in the technical specifications delimited by triple backticks.

    Use at most 5 words.

    Technical specifications: {class_name}
    """
    response = get_completion(prompt)

    # Append the response to the array
    responses_array.append(response)

    # Print the response only if it's different from the last printed response
    if response != last_printed_response:
        print(response)

        last_printed_response = response

# Now responses_array contains all the responses
# You can iterate over it to perform further actions

In [ ]:
from gtts import gTTS
from IPython.display import Audio
import time

# Iterate over the responses array
for response in responses_array:
    # Generate speech from the response
    tts = gTTS(response)

    # Save the speech as a .wav file
    output_file = 'output.wav'
    tts.save(output_file)

    # Play the audio
    display(Audio(output_file, autoplay=True))

    # Wait for the audio to finish playing
    time.sleep(4)  # Adjust the duration as needed